# Bitcoin (BTC) Price Prediction

This notebook was used to analyze the usefullness of Artificial Intelligence and Neural Networks specifically trying to predict the closing price of BTC.

In this first cell, there are defined some easily-switchable variables that might help achieving better results:

In [ ]:
import pandas as pd
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error

TARGET_COL = 'Close'
WINDOW_LEN = 5
TEST_SIZE = 0.2
ZERO_BASE = True
LSTM_NEURONS = 128
LSTM_SECOND_NEURONS = 64
EPOCHS = 25
BATCH_SIZE = 32
LOSS = 'mse'
DROPOUT = 0.2
OPTIMIZER = 'adam'

## Dataset 
I searched for several datasets of Bitcoin (BTC) prices, and the most complete I found have the following columns:
> \[ Date, Open Price, High Price, Low Price, Close Price, Volume \]

The Volume can be found in the currency (USD) or in BTC itself. Some also have Adjusted Close or Weighted Price.

The variable _dataset_ is a Pandas' DataFrame object, that holds the pure data coming out of the file, with irrelevant columns dropped.

In this particular case, the data have a frequency of 24h (with Date and Adj Close columns dropped):

> \[ \[ Open, High, Low, Close, Volume \], ... \]

In [ ]:
dataset = pd.read_csv("BTC-USD.csv")
dataset = dataset.drop(columns=['Date', 'Adj Close'])
print(dataset)

## Functions

Next some functions that help in the data processing, division and visualization

In [ ]:
# Split the df in train and test datasets
# default test_size is 20%
def train_test_split(df, test_size=0.2):
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data

In [ ]:
# Plot both train and test data
def line_plot(line1, line2, label1=None, label2=None, title='', lw=2):
    fig, ax = plt.subplots(1, figsize=(13, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('price [USD]', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16)

In [ ]:
# data normalisation, so they are all in the same scale
def normalise_zero_base(df):
    return df / df.iloc[0] - 1

def normalise_min_max(df):
    return (df - df.min()) / (data.max() - df.min())

In [ ]:
# arrange the dataset to the LSTM layer
# returns a 3D array
def extract_window_data(df, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [ ]:
# calls all above functions, returning the train and test datasets, ready to use
def prepare_data(df, target_col, window_len=5, zero_base=True, test_size=0.2):
    train_data, test_data = train_test_split(df, test_size=test_size)
    X_train = extract_window_data(train_data, window_len, zero_base)
    X_test = extract_window_data(test_data, window_len, zero_base)
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values
    if zero_base:
        y_train = y_train / train_data[target_col][:-window_len].values - 1
        y_test = y_test / test_data[target_col][:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test

In [ ]:
# builds and returns the NN model
def build_lstm_model(input_data, output_size, neurons=100, second_neurons=50, activ_func='linear',
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(LSTM(second_neurons))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [ ]:
train, test, X_train, X_test, y_train, y_test = prepare_data(dataset, TARGET_COL,window_len=WINDOW_LEN, zero_base=ZERO_BASE, test_size=TEST_SIZE)

print(X_train.shape)

## Model

Now it's time to build definitely the model, and train it with the train data we separated from the whole dataset.

You will see evolution through epochs of the processing and the loss value, hopefully dropping.

In [ ]:
model = build_lstm_model(X_train, output_size=1, neurons=LSTM_NEURONS,                                  second_neurons=LSTM_SECOND_NEURONS, dropout=DROPOUT,                            loss=LOSS, optimizer=OPTIMIZER)

history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,                         verbose=1, shuffle=True)

## Results

Here you can analyze the results.
The first value is the loss and below a plot is shown with the pairs (True Value, Prediction). The closer the dots to the x=y line, the better, because it means that the model predicted the same value as the real.

In [ ]:
targets = test[TARGET_COL][WINDOW_LEN:]
preds = model.predict(X_test).squeeze()
print(mean_absolute_error(preds, y_test))

test_predictions = model.predict(X_test).flatten()

plt.scatter(y_test, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

#preds = test[TARGET_COL].values[:-WINDOW_LEN] * (preds + 1)
#preds = pd.Series(index=targets.index, data=preds)
#line_plot(targets, preds, 'actual', 'prediction', lw=3)